In [4]:
import socket
import threading
import sys
import time
import pickle

local = '169.254.240.163', 50001
con = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
con.bind(local)
dest = '169.254.240.163', 50000


class tcp_header:
    sequence_number = None
    acknowledgement_number = None
    syn = False
    ack = False
    fin = False
    rst = False
    file = None

    def _init_(self, syn, sequence_number, file, fin):
        self.syn = syn
        self.sequence_number = sequence_number
        self.file = file
        self.fin = fin


class table():
    buffer_list = []
    tcpHeader = tcp_header


count = 100


def End_connection():
    print("sending confirmation ...")
    connection_object = tcp_header(False, None, None, True)
    data_byte = pickle.dumps(connection_object)
    con.sendto(data_byte, server)


def Wait_confirmation():
    global count
    print("Waiting for confirmation ...")
    data, server = con.recvfrom(2048)
    table.tcpHeader = pickle.loads(data)
    while table.tcpHeader.acknowledgement_number != count - 99:
        print("Broadcasting given ...")
        time.sleep(2)
        connection_object = tcp_header(False, count - 100, table.buffer_list[-1], False)
        data_byte = pickle.dumps(connection_object)
        con.sendto(data_byte, server)
        print("Waiting for confirmation ...")
        data, server = con.recvfrom(2048)
        table.tcpHeader = tcpHeader
        table.tcpHeader = pickle.loads(data)


def Send_file(buffer):
    global count
    connection_object = tcp_header(False, count, buffer, False)
    table.buffer_list.append(connection_object)
    data_byte = pickle.dumps(connection_object)
    con.sendto(data_byte, server)
    count = count + 100
    print("Sending file ...")
    buffer = None
    time.sleep(0.3)
    Wait_confirmation()
    return buffer


def Split_file():
    buffer_size = 64000
    f = open("submit.txt", "rb")
    buffer = f.read(buffer_size)
    while buffer:
        Send_file(buffer)
        buffer = f.read(buffer_size)
    End_connection()


try:
    con.settimeout(5)
    connection_object = tcp_header(True, None, None, False)
    data_string = pickle.dumps(connection_object)
    print("Starting connection")
    con.sendto(data_string, dest)
    print("Waiting for connection confirmation")
    data, server = con.recvfrom(2048)
    tcpHeader = pickle.loads(data)
    if tcpHeader.syn and tcpHeader.ack:
        Split_file()
except socket.timeout:
    print("TIMEOUT, ending connection ...")

OSError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions